In [15]:
import pandas as pd

In [16]:
revealed_targets = pd.read_csv('downloads/example_test_files/revealed_targets.csv')
test = pd.read_csv('downloads/example_test_files/test.csv')

In [17]:
revealed_targets['datetime'] = pd.to_datetime(revealed_targets['datetime'])
test['prediction_datetime'] = pd.to_datetime(test['prediction_datetime'])

In [18]:
def naive_predict(test_row, revealed_targets):
    last_known = revealed_targets[(revealed_targets.data_block_id == test_row.data_block_id) &
                                  (revealed_targets.is_consumption == test_row.is_consumption) &
                                  (revealed_targets.prediction_unit_id == test_row.prediction_unit_id) &
                                  (revealed_targets.datetime.map(lambda dt: dt.time()) == test_row.prediction_datetime.time() )]
    if len(last_known) == 0:
        last_known = 0
    else:
        last_known = last_known.target.mean()
    return last_known

def naive_predict_batch(test_batch, revealed_targets):
    target_series = test_batch.apply(lambda test_row: naive_predict(test_row, revealed_targets), axis=1)
    target_series.name = 'target'
    return pd.concat([test_batch[['row_id', 'data_block_id']], target_series], axis=1)

In [19]:
predicts = naive_predict_batch(test, revealed_targets)
predicts

,row_id,data_block_id,target
0,2005872,634,2.675
1,2005873,634,471.887
2,2005874,634,0.000
3,2005875,634,5.414
4,2005876,634,13.899
...,...,...,...
12475,2018347,637,188.167
12476,2018348,637,0.000
12477,2018349,637,31.484
12478,2018350,637,0.000


In [20]:
predicts.describe()

,row_id,data_block_id,target
count,1.248000e+04,12480.000000,12480.000000
mean,2.012112e+06,635.500000,387.985572
std,3.602810e+03,1.118079,1045.722829
min,2.005872e+06,634.000000,0.000000
25%,2.008992e+06,634.750000,10.322750
50%,2.012112e+06,635.500000,64.640500
75%,2.015231e+06,636.250000,300.997250
max,2.018351e+06,637.000000,11146.496000
